# Spatial Weights Matrix Construction
Master thesis of Nikolai Popov, MAE-2025

In [1]:
# Libraries import
import numpy as np # for matrices/vectors
import warnings # to supress warninings
warnings.simplefilter(action='ignore', category=Warning)
from scipy.spatial import distance_matrix # for distance matrix creation
import geopandas as gpd # to to read shape files
from sklearn.neighbors import NearestNeighbors # for exception in spatial weights
from scipy.sparse import identity, kron, csr_matrix, save_npz # for memory efficient sparse matrix

Load the shape file from QGIS with already proper projection for Russia in meters.

In [2]:
# Read the shapefile (will automatically detect .shp + associated .dbf, .shx, etc.)
gdf = gpd.read_file("C:/Users/Popov/Documents/Research/Volchkova_thesis/Data/GEO/firms_points_meters_shape.shp")
gdf.head(2)

,INN,Year,okveds,Capital,Labor,Output,Post_d,Treated_d,Latitude,Longitude,geometry
0,2.435001e+09,2014,"['01.11', '01.11', '01.41', '01.42', '01.61', ...",423047000.0,546,316176000.0,True,True,56.552422,93.047783,POINT (-714627.087 6626570.316)
1,2.427001e+09,2014,"['01.11', '01.11', '01.30', '01.41', '01.42']",193875000.0,568,196375000.0,True,True,55.837950,90.999293,POINT (-849651.822 6570463.818)


In [3]:
# Check the current CRS - check the projection
print(gdf.crs)  # Should show ESRI:102012 or its full WKT

PROJCS["Asia_Lambert_Conformal_Conic",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["standard_parallel_1",30],PARAMETER["standard_parallel_2",62],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102012"]]


"ESRI","102012" - the proper projection.

Now to matrix computation.

## W_N

In [4]:
# Get point coordinates (via Point geometry)
coords = np.array([[geom.x, geom.y] for geom in gdf.geometry])

# Use Euclidean distance with threshold = 300 km = 300,000 meters
from scipy.spatial import distance_matrix
dist_matrix = distance_matrix(coords, coords)
W = (dist_matrix < 300000).astype(int)
np.fill_diagonal(W, 0)
W

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
# Total number of firms
total_firms = W.shape[0]

# Number of firms with no neighbors
zero_rows = np.sum(W.sum(axis=1) == 0)

# Percentage
percent_isolated = (zero_rows / total_firms) * 100

print(f"Number of all-zero rows: {zero_rows}")
print(f"Total firms: {total_firms}")
print(f"Percentage of isolated firms: {percent_isolated:.2f}%")

# Total number of nonzero weights
nonzeros = W.sum()

# Average number of neighbors per location
avg_neighbors = nonzeros / W.shape[0]
print(f"Average neighbors per location: {avg_neighbors:.2f}")

Number of all-zero rows: 6
Total firms: 8492
Percentage of isolated firms: 0.07%
Average neighbors per location: 619.89


Well, zero rows are prohibited - will be might be a problem for linear algebra operations.

In [6]:
# Step 1: Get a boolean mask of rows with no neighbors
no_neighbors_mask = W.sum(axis=1) == 0

# Step 2: Use it to filter the original GeoDataFrame
inn_no_neighbors = gdf.loc[no_neighbors_mask, 'INN']

# Step 3: Display
print(inn_no_neighbors)

340     8.703001e+09
1336    8.701003e+09
2712    1.420001e+09
3512    8.704004e+09
4044    8.802002e+09
4313    1.416001e+09
Name: INN, dtype: float64


This is a list of INNs of isolated firms. There is only a very small percentage of them, thus, as an exception, let's use K-nearest neighbors (KNN) to assign neighbors to these firms. As these firms are isolated, let's go with 10 nearest neighbors.

In [7]:
# Step 1: Identify isolated firm indices
isolated_indices = gdf[gdf['INN'].isin(inn_no_neighbors)].index.to_numpy()

# Step 2: Fit KNN on all coordinates
from sklearn.neighbors import NearestNeighbors

k = 10
nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(coords)
distances, indices = nbrs.kneighbors(coords)

# Step 3: Add KNN weights only for isolated firms

# For each isolated firm, assign 10 nearest neighbors using KNN.
# To maintain symmetry in the spatial weights matrix, also add reciprocal links:
# if firm i considers firm j a neighbor, then firm j also considers firm i a neighbor.

for i in isolated_indices:
    for j in indices[i][1:]:  # skip self
        W[i, j] = 1          # firm i (isolated) considers firm j (any firm) a neighbor
        W[j, i] = 1          # firm j also considers firm i a neighbor

In [8]:
# Checking the symmetry
is_symmetric = np.array_equal(W, W.T)
print(f"Matrix is symmetric: {is_symmetric}")

Matrix is symmetric: True


In [9]:
# Convert to CSR format (for space efficiency)
W_N_sparse = csr_matrix(W)

# Save to disk
save_npz("C:/Users/Popov/Documents/Research/Volchkova_thesis/Data/Cleaned_datasets/W_N_sparse.npz", W_N_sparse)

## W_NT

In [11]:
# Convert W to sparse format
W_sparse = csr_matrix(W)

# Construct Kronecker product: W_NT = I_T ⊗ W
T = 6
W_NT_sparse = kron(identity(T, format='csr'), W_sparse, format='csr')

# Save to disk
save_npz("C:/Users/Popov/Documents/Research/Volchkova_thesis/Data/Cleaned_datasets/W_NT_sparse.npz", W_NT_sparse)

# Report shape and sparsity
print(f"Shape: {W_NT_sparse.shape}")
print(f"Non-zero entries: {W_NT_sparse.nnz}")

Shape: (50952, 50952)
Non-zero entries: 31585344
